In [ ]:
#@title Code - Please run once!
!pip install rdflib
!pip install PyGithub
!pip install rdflib
import ipywidgets as widgets
from IPython.display import display, clear_output
import getpass
from urllib.request import urlretrieve
import base64

url = "https://raw.githubusercontent.com/Mat-O-Lab/MapToMethod/main/maptomethod.py"
file_name = url.split('/')[-1]

import os.path
if not os.path.isfile(file_name):
  urlretrieve(url,file_name) 
import maptomethod

class MapDialog():
    def __init__(self, csv_meta_url='', method_url=''):
      self.csv_meta_url = csv_meta_url
      self.method_url = method_url
      self.methods=maptomethod.get_methods()
    def _create_initial_widgets(self):
      self.meta_url_widget=widgets.Text(
          value='',
          placeholder='put ur url to a *-metadata.json here',
          description='Url:',
          disabled=False
          )
      # self.uploader = widgets.FileUpload(accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
      #                                    multiple=False,  # True to accept multiple files upload else False
      #                                    description='Upload'
      #                                    )
      #self.clear_button = widgets.Button(description='Clear!', layout=widgets.Layout(width='100px')); 
      self.file_dialog= widgets.HBox([widgets.Label(value="File:"), self.meta_url_widget]) #,self.uploader,self.clear_button])
      #self.clear_button.on_click(self._on_clear)
      # self.data_type_sel=widgets.Box(
      #     [
      #       widgets.Label(value='primary or secondary data:'),
      #       widgets.RadioButtons(
      #           options=[
      #                   'primary - the data describes one experiment with one specimen',
      #                   'secondary - data table with results of multiple experiments',
      #                   ],
      #           layout={'width': 'max-content'}
      #           )
      #       ]
      #   )
      self.method_sel = widgets.Dropdown(
          options=list(self.methods.keys()),
          #value=list(methods.items())[0][1],
          description='method:',
          disabled=False,
          )
      self.out = widgets.Output()  # this is the output widget in which the df is displayed
      self.start_mapping_button = widgets.Button(description='Start Mapping!', layout=widgets.Layout(width='200px')); 
      self.start_mapping_button.on_click(self._start_mapping)
    def _on_clear(self,button):
      self.uploader.value.clear()
      self.uploader._counter = 0
    def _start_mapping(self,button):
      # "linking function with output"
      with self.out:
      # what happens when we press the button
        clear_output()
        if not (self.meta_url_widget.value or self.uploader.value.keys()):
          print('pls upload a file first or insert a url')
          return
        if self.meta_url_widget.value:
          self.csv_meta_url=self.meta_url_widget.value
          file_name=self.csv_meta_url.split('/')[-1]
        else:
          input_file=self.uploader.value[list(self.uploader.value.keys())[0]]
          self.csv_meta_url=input_file['metadata']['name']
          file_name = input_file['metadata']['name']
          self.file_data = input_file['content']
        self.method_url=self.methods[self.method_sel.value]     
        self.mapper=maptomethod.Mapper(self.csv_meta_url,self.method_url)
        self.methode_ices=self.mapper.ices
        self.data_info=self.mapper.info_lines
        info_lines=[(value['text'],id) for id, value in self.data_info.items()]
        info_lines.insert(0,('None',None))
        self.map_elements=list()
        for id, url in self.methode_ices.items():
          self.map_elements.append(widgets.HBox([
                                            widgets.Label(id),
                                            widgets.Dropdown(
                                                options=info_lines,
                                                disabled=False,)
                                            ]))
        display(widgets.VBox(self.map_elements))
        self.create_mapping_button = widgets.Button(description='Create Mapping!', layout=widgets.Layout(width='200px'));
        buttons= widgets.HBox([self.create_mapping_button,])
        self.create_mapping_button.on_click(self._create_mapping)
        display(buttons)
    def _create_mapping(self,button):
      # "linking function with output"
      with self.out:
        clear_output()
        map_list=[element.children for element in self.map_elements]
        map_list=[(widget_label.value,select.value) for (widget_label,select) in map_list if select.value]
        self.mapper.maplist=map_list
        
        mapping_filename, mapping_data=self.mapper.to_yaml()
        print(mapping_data)      
        res = mapping_data
        b64 = base64.b64encode(res.encode())
        payload = b64.decode()
        button_html = '''<html>
        <head>
        <meta name="viewport" content="width=device-width, initial-scale=1">
        </head>
        <body>
        <a download="{filename}" href="data:text/json;base64,{payload}" download>
        <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">{title}</button>
        </a>
        </body>
        </html>
        '''
        download_button = button_html.format(title='Download File',payload=payload,filename=mapping_filename)
        display(widgets.HTML(download_button))
      
    def display_widgets(self):
        self._create_initial_widgets()
        display(widgets.VBox(
                    [
                      self.file_dialog,
                      #self.data_type_sel,
                      self.method_sel,
                      self.start_mapping_button,
                      self.out
                    ]
                )
        )


In [3]:
#@title Dialog - Run Cell to begin
dialog = MapDialog()
dialog.display_widgets()